# CIGALE catalogue preparation on the AKARI-SEP field.

This notebook prepares the catalogues that will be analysed by CIGALE for SED fitting and physical parameter estimation.

We are also making a further catalogue including recent MeerKat data.

In [1]:
import numpy as np
import os
os.environ['LOG_LEVEL'] = 'INFO'

from astropy.table import Table, join


from herschelhelp.filters import correct_galactic_extinction
from herschelhelp.external import convert_table_for_cigale

from collections import OrderedDict

In [2]:
SUFFIX = '20180221'

master_catalogue = Table.read("../../dmu32/dmu32_AKARI-SEP/data/AKARI-SEP_{}_cigale.fits".format(SUFFIX))

In [3]:
len(master_catalogue)

844172

# Best sources

Define a good far-IR measurement as:
- an existing flux in the band;
- the flag from XID+ must not be set;
- the signal to noise ratio must be over 2.

In [4]:
good = {}
for band in ['pacs_green', 'pacs_red', 'spire_250', 'spire_350', 'spire_500']:
    good[band] = (~np.isnan(master_catalogue['f_{}'.format(band)]) & 
                                   ~master_catalogue['flag_{}'.format(band)])
    good[band][good[band]] &= (master_catalogue[good[band]]['f_{}'.format(band)] /
                              master_catalogue[good[band]]['ferr_{}'.format(band)] >= 2)

We will keep only sources with at leat 2 good far-IR measurements (we may actually use less sources are not all may have a redshift).

In [5]:
combined_good = np.sum(list(good.values()), axis=0) >= 2

In [6]:
print("Number of good sources: {}".format(np.sum(combined_good)))

Number of good sources: 946


In [7]:
# Only sources with at least two optical and at least two near infrared detections
optnir = ((master_catalogue['flag_optnir_det'] == 3) 
          | (master_catalogue['flag_optnir_det'] == 7))

# Main catalogue for CIGALE

In [8]:
best_catalogue = master_catalogue[combined_good].copy()


In [9]:
best_catalogue = best_catalogue[~np.isnan(best_catalogue["redshift"])]

In [10]:
# Correction for galactic extinction
best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [11]:
# Convertion to CIGALE format
best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

## Band selection

We want to use only one filter for similar bands. We define an order of preference and set to NaN the flux in the lower prefered bands when a prefered band is available. Some band may have a 0 flux, we set there values to NaN.

In [12]:
u_bands = []
g_bands = ["decam_g"]
r_bands = ["decam_r"]
i_bands = ["decam_i"]
z_bands = ["decam_z"]
y_bands = ["decam_y"]

def remove_unneeded_fluxes(list_of_bands):
    for band_idx, band in enumerate(list_of_bands[:-1]):
        mask = ~np.isnan(best_catalogue[band])
        for lower_band in list_of_bands[band_idx+1:]:
            best_catalogue[lower_band][mask] = np.nan
            best_catalogue["{}_err".format(lower_band)][mask] = np.nan

In [13]:
remove_unneeded_fluxes(u_bands)
remove_unneeded_fluxes(g_bands)
remove_unneeded_fluxes(r_bands)
remove_unneeded_fluxes(i_bands)
remove_unneeded_fluxes(z_bands)
remove_unneeded_fluxes(y_bands)

In [14]:
!mkdir data_tmp

mkdir: data_tmp: File exists


In [15]:
best_catalogue.write("data_tmp/AKARI-SEP_cigale_best_extcor_{}.fits".format(SUFFIX), overwrite=True)

# Catalogue using spectroscopic redshift

In [16]:
best_catalogue = master_catalogue[combined_good].copy()

In [17]:
best_catalogue.remove_column("redshift")
best_catalogue["zspec"].name = "redshift"

In [18]:
best_catalogue = best_catalogue[~np.isnan(best_catalogue["redshift"])]

In [19]:
print("Number of sources with z-spec: {}".format(len(best_catalogue)))

Number of sources with z-spec: 71


In [20]:
# Correction for galactic extinction
best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [21]:
# Convertion to CIGALE format
os.environ['LOG_LEVEL'] = 'INFO'
best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

In [22]:
remove_unneeded_fluxes(g_bands)
remove_unneeded_fluxes(u_bands)
remove_unneeded_fluxes(r_bands)
remove_unneeded_fluxes(i_bands)
remove_unneeded_fluxes(z_bands)
remove_unneeded_fluxes(y_bands)

In [23]:
best_catalogue.write("data_tmp/AKARI-SEP_cigale_best_extcor_zspec_{}.fits".format(SUFFIX), overwrite=True)

## MeerKat catalogue

In [24]:
meerkat=Table.read('../../dmu9/dmu9_MeerKat/data/adfs_meerkat_all_s9_matched_srcs_ver5.fits')

In [25]:
best_catalogue = master_catalogue[combined_good].copy()

In [26]:
meerkat[:5]

ID_S9,RA_S9,DEC_S9,TOTAL_FLUX_S9,E_TOTAL_FLUX_S9,PEAK_FLUX_S9,E_PEAK_FLUX_S9,ISL_RMS_S9,MATCH_ID,HELP_ID,RA_HELP,DEC_HELP,ZBEST,Z_PHOTO,E_Z_PHOTO,Z_SPEC,NRCs_S9,RC_S9
int32,float64,float64,float64,float64,float64,float64,float64,int32,bytes27,float64,float64,float64,float64,float64,float64,int16,bytes42
2,73.6874587111701,-52.87401454557761,3.0003349429645423e-05,8.05257766425511e-06,1.0697886712665846e-05,2.17915980779838e-06,2.106207148244721e-06,553025,HELP_J045444.881-525225.354,73.68700314051341,-52.873709421457704,0.3864355,0.3864355,0.10539615,nan,1,[2]
7,73.67599940758815,-52.80897480584811,4.245745096044256e-05,5.696126596292526e-06,2.403726760005785e-05,2.203890412591199e-06,2.0401498659339268e-06,484471,HELP_J045442.129-524831.962,73.67553619459312,-52.80887823188071,nan,nan,nan,nan,1,[7]
8,73.6911122603053,-53.09929881259075,1.2451530909212555e-05,5.108263627616426e-06,8.591369347374768e-06,2.278275684462228e-06,2.141669256161549e-06,347972,HELP_J045445.762-530557.482,73.69067543588658,-53.099300435340346,0.039888,0.026589284,0.007870829000000001,0.039888,1,[8]
9,73.68674172101788,-53.061034927872065,8.936688465253442e-06,3.1092906092600507e-06,1.1321505889260284e-05,2.065669598037972e-06,2.241039055661531e-06,994252,HELP_J045444.737-530339.749,73.6864021934515,-53.06104125020421,0.72683567,0.72683567,0.26864845,nan,1,[9]
10,73.65029964301772,-52.437255754196826,1.924403774353546e-05,5.6494271866775076e-06,1.3488011297328689e-05,2.5545346458872798e-06,2.3938555386848748e-06,236184,HELP_J045435.951-522615.173,73.64979592157016,-52.437548065655285,0.46191818,0.46191818,0.019458273,nan,1,[10]


In [27]:
imported_columns=OrderedDict({
    'HELP_ID':'help_id',
    'TOTAL_FLUX_S9':'f_meerkat_s9',
    'E_TOTAL_FLUX_S9':'ferr_meerkat_s9',
    'ZBEST':'meerkat_photoz',
})
meerkat=meerkat[list(imported_columns)]
for column in imported_columns:
    meerkat[column].name = imported_columns[column]

In [28]:
len(meerkat)

12877

In [29]:
best_catalogue[:5]

help_id,field,ra,dec,hp_idx,ebv,redshift,zspec,f_decam_g,ferr_decam_g,f_ap_decam_g,ferr_ap_decam_g,m_decam_g,merr_decam_g,m_ap_decam_g,merr_ap_decam_g,flag_decam_g,f_decam_r,ferr_decam_r,f_ap_decam_r,ferr_ap_decam_r,m_decam_r,merr_decam_r,m_ap_decam_r,merr_ap_decam_r,flag_decam_r,f_decam_i,ferr_decam_i,f_ap_decam_i,ferr_ap_decam_i,m_decam_i,merr_decam_i,m_ap_decam_i,merr_ap_decam_i,flag_decam_i,f_decam_z,ferr_decam_z,f_ap_decam_z,ferr_ap_decam_z,m_decam_z,merr_decam_z,m_ap_decam_z,merr_ap_decam_z,flag_decam_z,f_decam_y,ferr_decam_y,f_ap_decam_y,ferr_ap_decam_y,m_decam_y,merr_decam_y,m_ap_decam_y,merr_ap_decam_y,flag_decam_y,f_vista_j,ferr_vista_j,f_ap_vista_j,ferr_ap_vista_j,m_vista_j,merr_vista_j,m_ap_vista_j,merr_ap_vista_j,flag_vista_j,f_vista_h,ferr_vista_h,f_ap_vista_h,ferr_ap_vista_h,m_vista_h,merr_vista_h,m_ap_vista_h,merr_ap_vista_h,flag_vista_h,f_vista_ks,ferr_vista_ks,f_ap_vista_ks,ferr_ap_vista_ks,m_vista_ks,merr_vista_ks,m_ap_vista_ks,merr_ap_vista_ks,flag_vista_ks,f_irac_i1,ferr_irac_i1,f_ap_irac_i1,ferr_ap_irac_i1,m_irac_i1,merr_irac_i1,m_ap_irac_i1,merr_ap_irac_i1,flag_irac_i1,f_irac_i2,ferr_irac_i2,f_ap_irac_i2,ferr_ap_irac_i2,m_irac_i2,merr_irac_i2,m_ap_irac_i2,merr_ap_irac_i2,flag_irac_i2,f_mips_24,ferr_mips_24,flag_mips_24,f_pacs_green,ferr_pacs_green,flag_pacs_green,f_pacs_red,ferr_pacs_red,flag_pacs_red,f_spire_250,ferr_spire_250,flag_spire_250,f_spire_350,ferr_spire_350,flag_spire_350,f_spire_500,ferr_spire_500,flag_spire_500,stellarity,stellarity_origin,flag_cleaned,flag_merged,flag_gaia,flag_optnir_obs,flag_optnir_det,zspec_qual,zspec_association_flag
,,deg,deg,,,,,uJy,uJy,uJy,uJy,mag,mag,mag,mag,,uJy,uJy,uJy,uJy,mag,mag,mag,mag,,uJy,uJy,uJy,uJy,mag,mag,mag,mag,,uJy,uJy,uJy,uJy,mag,mag,mag,mag,,uJy,uJy,uJy,uJy,mag,mag,mag,mag,,uJy,uJy,uJy,uJy,mag,mag,mag,mag,,uJy,uJy,uJy,uJy,mag,mag,mag,mag,,uJy,uJy,uJy,uJy,mag,mag,mag,mag,,uJy,uJy,uJy,uJy,mag,mag,mag,mag,,uJy,uJy,uJy,uJy,mag,mag,mag,mag,,uJy,uJy,,uJy,uJy,,uJy,uJy,,uJy,uJy,,uJy,uJy,,uJy,uJy,,,,,,,,,,
bytes27,bytes18,float64,float64,int64,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,bool,float32,float32,float32,float32,float32,float32,float32,float32,bool,float32,float32,float32,float32,float32,float32,float32,float32,bool,float32,float32,float32,float32,float32,float32,float32,float32,bool,float32,float32,float32,float32,float32,float32,float32,float32,bool,float32,float32,float32,float32,float32,float32,float32,float32,bool,float32,float32,bool,float32,float32,bool,float32,float32,bool,float32,float32,bool,float32,float32,bool,float32,float32,bool,float64,bytes20,bool,bool,int64,int64,int64,int64,bool
HELP_J043743.047-541546.529,AKARI-SEP,69.42936156056442,-54.26292467587036,556165883,0.005216917337193211,nan,nan,1.1602380903150522,0.2087293811479051,1.0023778377777695,0.12966795457713476,23.738632202148438,0.1953263282775879,23.89742136001587,0.14045122265815735,False,1.9693890208952047,0.22847464966012418,1.9488443098278312,0.13654527652564724,23.16417121887207,0.12595947086811066,23.175557136535645,0.0760718286037445,False,2.2630519261799336,0.4343343840479813,2.554178941192882,0.24907195271175267,23.013263702392578,0.20837903022766113,22.881871700286865,0.10587607324123383,False,4.426770795468066,0.7643476754630564,4.347653160843821,0.4314072593419604,22.28478240966797,0.18746846914291382,22.304362773895264,0.10773501545190811,False,5.242857136142722,2.4113265347221673,3.491993489184114,1.3888638898821069,22.1010799407959,0.4993583559989929,22.542316436767578,0.4318277835845947,False,nan,nan,nan,nan,nan,nan,nan,nan,False,nan,nan,nan,nan,nan,nan,nan,nan,False,nan,nan,nan,nan,nan,nan,nan,nan,False,46.757805,1.2191161,43.877453,0.9411782,

In [30]:
len(best_catalogue)

946

In [31]:
len(set(meerkat['help_id']).intersection(set(best_catalogue['help_id'])))

608

In [32]:
best_catalogue=join(best_catalogue, meerkat, join_type='left')

In [33]:
best_catalogue['redshift']=best_catalogue['meerkat_photoz']

In [34]:
best_catalogue.write("data_tmp/AKARI-SEP_cigale_best_extcor_meerkat_{}.fits".format(SUFFIX), overwrite=True)